In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. 문서 준비
documents = [
    "한화이글스는 1986년에 창단되었습니다.",
    "홈구장은 한화생명이글스파크입니다.",
    "대전광역시를 연고지로 합니다.",
    "정근우, 김태균 같은 선수를 배출했습니다.",
    "기아타이거즈는 광주를 연고지로 합니다."
]

In [3]:
# 2. 모델 로딩
model = SentenceTransformer("intfloat/multilingual-e5-small")

# 3. 문서 임베딩
doc_embeddings = model.encode([f"passage: {d}" for d in documents], convert_to_numpy=True)

# 4. FAISS 인덱스 구성
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [6]:
# 5. 질문 입력 및 검색
query = "한화이글스는 어디 연고지야?"
query_embedding = model.encode(f"query: {query}", convert_to_numpy=True).reshape(1, -1)

D, I = index.search(query_embedding, 3)
for rank, idx in enumerate(I[0]):
    print(f"[{rank+1}위] {documents[idx]} (거리: {D[0][rank]:.4f})")

[1위] 한화이글스는 1986년에 창단되었습니다. (거리: 0.2613)
[2위] 대전광역시를 연고지로 합니다. (거리: 0.3223)
[3위] 홈구장은 한화생명이글스파크입니다. (거리: 0.3336)


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. 문서 준비
documents = [
    "한화이글스는 1986년에 창단되었습니다.",
    "홈구장은 한화생명이글스파크입니다.",
    "대전광역시를 연고지로 합니다.",
    "정근우, 김태균 같은 선수를 배출했습니다.",
    "기아타이거즈는 광주를 연고지로 합니다."
]

# 2. 모델 로딩
model = SentenceTransformer("intfloat/multilingual-e5-small")

# 3. 문서 임베딩
doc_embeddings = model.encode([f"passage: {d}" for d in documents], convert_to_numpy=True)

# 4. FAISS 인덱스 구성
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# 5. 질문 입력 및 검색
query = "한화이글스는 어디 연고지야?"
query_embedding = model.encode(f"query: {query}", convert_to_numpy=True).reshape(1, -1)

D, I = index.search(query_embedding, 3)
for rank, idx in enumerate(I[0]):
    print(f"[{rank+1}위] {documents[idx]} (거리: {D[0][rank]:.4f})")

In [ ]:
import re
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 1. Markdown 형식의 문서 입력
md_text = """
### MOA 3150 폐지된 적용사례의 적용
MOA 3130 또는 MOA 3140에 따라 사용이 승인된 적용사례는 폐지된 이후에도 그 가동중시험 계획 기간 동안에는 계속 사용할 수 있다.

### MOA 3160 시험 및 점검 절차서
시험 및 점검은 문서화된 절차서에 따라 수행해야 한다. 절차서에는 발전사업자가 제시한 기준값과 허용기준이 있어야 한다.

### MOA 3200 관리요건
1. 가동중시험 계획서는 규제기관에 제출하여야 한다.  
2. 시험계획에 포함되는 기기의 선정은 규제기관의 검토를 받아야 한다.  
3. 이 기술기준의 가동중시험 요건의 적용은 규제기관의 등급분류 기준에 따라야 한다.  
4. 모든 적용사례의 사용은 규제기관의 승인을 받아야 한다.  
5. 이미 승인된 적용사례의 개정본은 규제기관의 허가 후에 기존 적용사례를 대신하여 사용할 수 있다.  
6. 시험 및 점검계획은 다음과 같은 가동중시험 기술기준판 및 추록의 요건을 만족하여야 한다.  
   - **(가) 가동전시험**:  
     가동전시험 계획은 해당시설의 건설허가일 36개월 전에 규제기관에서 채택한 이 기술기준의 최신 기준판 및 추록 또는 가능하면 표준설계인가에서 참조한 이 규정의 기준판 및 추록을 만족하여야 한다. 대안으로 가동전시험 계획은 규제기관에서 정한 후속 기준판 및 추록을 만족하여야 한다. 이 후속 기준판 및 추록의 특정부분 만을 적용할 수도 있으나, 이를 위하여서는 특정부분의 관련요건을 모두 만족하여야 한다.  
   - **(나) 초기 가동중시험**:  
     초기 가동중시험계획은 운영허가 발급 12개월 전에 규제기관에서 채택한 이 기술기준의 최신 기준판 및 추록을 만족하여야 한다. 대안으로 가동중시험 계획은 규제기관에서 정한 후속 기준판 및 추록을 만족하여야 한다. 이 후속 기준판 및 추록의 특정부분 만을 적용할 수도 있으나, 이를 위하여서는 특정부분의 관련요건을 모두 만족하여야 한다.  
   - **(다) 후속 가동중시험**:  
     후속 가동중시험계획은 가동중시험 시작 12개월 전에 규제기관에서 채택한 이 기술기준의 기준판 및 추록 또는 규제기관에서 정한 후속 기준판 및 추록을 만족하여야 한다. 이 후속 기준판 및 추록의 특정부분 만을 적용할 수도 있으나, 이를 위하여서는 특정부분의 관련요건을 모두 만족하여야 한다.

### MOA 3300 시정조치
보수/교체가 필요한 시정조치는 가능하면 KEPIC MI에 따라 수행하여야 한다. 다른 시정조치는 발전사업자의 품질보증 계획에 따라 수행하여야 한다.

"""

# 2. Chunking (### 기준)
chunks = re.split(r"\n(?=### )", md_text.strip())
documents = [chunk.strip() for chunk in chunks if chunk.strip()]

# 3. 모델 로딩 및 임베딩
model = SentenceTransformer("intfloat/multilingual-e5-small")
doc_embeddings = model.encode([f"passage: {d}" for d in documents], convert_to_numpy=True)

# 4. FAISS 인덱스 구성
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# 5. 질문 입력 후 검색
query = "MOA 3200 관리요건에 대해 알려줘"
query_embedding = model.encode(f"query: {query}", convert_to_numpy=True).reshape(1, -1)

D, I = index.search(query_embedding, 3)
for rank, idx in enumerate(I[0]):
    print(f"[{rank+1}위] {documents[idx]} (거리: {D[0][rank]:.4f})")


[1위] ### MOA 3200 관리요건
1. 가동중시험 계획서는 규제기관에 제출하여야 한다.  
2. 시험계획에 포함되는 기기의 선정은 규제기관의 검토를 받아야 한다.  
3. 이 기술기준의 가동중시험 요건의 적용은 규제기관의 등급분류 기준에 따라야 한다.  
4. 모든 적용사례의 사용은 규제기관의 승인을 받아야 한다.  
5. 이미 승인된 적용사례의 개정본은 규제기관의 허가 후에 기존 적용사례를 대신하여 사용할 수 있다.  
6. 시험 및 점검계획은 다음과 같은 가동중시험 기술기준판 및 추록의 요건을 만족하여야 한다.  
   - **(가) 가동전시험**:  
     가동전시험 계획은 해당시설의 건설허가일 36개월 전에 규제기관에서 채택한 이 기술기준의 최신 기준판 및 추록 또는 가능하면 표준설계인가에서 참조한 이 규정의 기준판 및 추록을 만족하여야 한다. 대안으로 가동전시험 계획은 규제기관에서 정한 후속 기준판 및 추록을 만족하여야 한다. 이 후속 기준판 및 추록의 특정부분 만을 적용할 수도 있으나, 이를 위하여서는 특정부분의 관련요건을 모두 만족하여야 한다.  
   - **(나) 초기 가동중시험**:  
     초기 가동중시험계획은 운영허가 발급 12개월 전에 규제기관에서 채택한 이 기술기준의 최신 기준판 및 추록을 만족하여야 한다. 대안으로 가동중시험 계획은 규제기관에서 정한 후속 기준판 및 추록을 만족하여야 한다. 이 후속 기준판 및 추록의 특정부분 만을 적용할 수도 있으나, 이를 위하여서는 특정부분의 관련요건을 모두 만족하여야 한다.  
   - **(다) 후속 가동중시험**:  
     후속 가동중시험계획은 가동중시험 시작 12개월 전에 규제기관에서 채택한 이 기술기준의 기준판 및 추록 또는 규제기관에서 정한 후속 기준판 및 추록을 만족하여야 한다. 이 후속 기준판 및 추록의 특정부분 만을 적용할 수도 있으나, 이를 위하여서는 특정부분의 관련요건을 모두 만족하여야 한다. (거리: 0.1983)
[2위] ### MOA 3150 폐지된 적용사